In [ ]:
import pandas as pd
import nltk
import warnings
import re
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/ML/sms_spam.csv")
y = data['type']    # spam or ham - target
X = data['text']    # message text - input data
# print(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
# print('X_train:', X_train.head())
import string
for ch in string.punctuation:
    X_train = X_train.str.replace(ch, "")  # remove punctuation

In [ ]:
X_train = X_train.str.lower()  # make lowercase
# print('X_train_lower:', X_train)
from nltk.corpus import stopwords
nltk.download('stopwords')
noise = stopwords.words('english')
"""DO THIS TO REMOVE NOISE FROM A STRING: """   # removing stop_words
for st in noise:
    X_train = X_train.apply(lambda x: ' '.join([word for word in x.split()
                                                if word not in noise]))
# print(X_train.head(10))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
X_train = X_train.apply(word_tokenize)  # tokenizing a dataframe into unigrams
# print(X_train.head())
porter = nltk.PorterStemmer()  # lemmatization/stemming
X_train = X_train.apply(lambda x: [porter.stem(y) for y in x])  # stemming each word in str
X_train = X_train.apply(lambda x: ' '.join(x))   # a column of lists is put back to str for vectorizer
# print(X_train.head(10))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
"""VECTORIZATION"""
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
vectorizer_metrics_df = pd.DataFrame(columns=['n-grams', 'Class', 'Precision', 'Recall', 'F1-Score', 'Support'])
# will be used later for metrics

In [ ]:
'''CHOOSING THE NUMBER OF N_GRAMS'''
for i in range (1, 5):
    smart_vectorizer = CountVectorizer(ngram_range=(1, i))  # 1, 2 - bigrams. 1, 3 - trigrams, 1, 1 -unig.
    smart_vectorized_x_train = smart_vectorizer.fit_transform(X_train)
# classification

    from sklearn.naive_bayes import MultinomialNB  # naive Bayes classifier
    clf = MultinomialNB()
    clf.fit(smart_vectorized_x_train, y_train)   # fit data to classifier
    smart_vectorized_x_test = smart_vectorizer.transform(X_test)
    pred = clf.predict(smart_vectorized_x_test)
    report_count = classification_report(y_test, pred)

    lines = report_count.strip().split('\n')
    """the following code is used to concat all different report DFs into one by tag spam"""

    for line in lines[2:4]:
        tokens = re.split(r'\s+', line.strip())
        class_name, precision, recall, f1_score, support = tokens[0],\
                                                           float(tokens[1]),\
                                                           float(tokens[2]), \
                                                           float(tokens[3]), \
                                                           int(tokens[4])

        if class_name in ['spam']:
            vectorizer_metrics_df = vectorizer_metrics_df._append({
                'n-grams': i,
                'Class': class_name,
                'Precision': precision,
                'Recall': recall,
                'F1-Score': f1_score,
                'Support': support
            }, ignore_index=True)

print('metrics for different n_grams within Count Vectorizer: ', vectorizer_metrics_df, sep='\n', end='\n')
# appropriate number of n_grams is 2

metrics for different n_grams within Count Vectorizer: 
  n-grams Class  Precision  Recall  F1-Score Support
0       1  spam       0.93    0.87      0.90     211
1       2  spam       0.97    0.82      0.89     211
2       3  spam       0.97    0.82      0.89     211
3       4  spam       0.98    0.81      0.88     211


In [ ]:
smart_vectorizer = CountVectorizer(ngram_range=(1, 2))
smart_vectorized_x_train = smart_vectorizer.fit_transform(X_train)
# classification
from sklearn.naive_bayes import MultinomialNB  # naive Bayes classifier
clf = MultinomialNB()
clf.fit(smart_vectorized_x_train, y_train)   # fit data to classifier
smart_vectorized_x_test = smart_vectorizer.transform(X_test)
pred_n_grams = clf.predict(smart_vectorized_x_test)

In [ ]:
x_test = pd.Series({1:'Congratulations! You won a $1,000 Walmart gift card. Go to http://bit.ly/123456 to claim reward',
                    2:'cvut university informs you about your enrollment'})
smart_vectorized_x_test_1 = smart_vectorizer.transform(x_test)
pred_n_grams_1 = clf.predict(smart_vectorized_x_test_1)
df = pd.DataFrame({'text' : x_test.to_list(), 'label': pred_n_grams_1})
df

,text,label
0,"Congratulations! You won a $1,000 Walmart gift...",spam
1,cvut university informs you about your enrollment,ham


In [ ]:
x_test[1]

'you recieved 1000000 dollars click here to redeem'

In [ ]:
df = pd.DataFrame({'text' : X_test.to_list(), 'label': pred_n_grams})
df[df.label == 'spam']

,text,label
16,Hey I am really horny want to chat or see me n...,spam
20,Adult 18 Content Your video will be with you s...,spam
39,FreeMsg: Claim ur 250 SMS messages-Text OK to ...,spam
59,74355 XMAS iscoming & ur awarded either £500 C...,spam
74,FREE MESSAGE Activate your 500 FREE Text Messa...,spam
...,...,...
1612,Call FREEPHONE 0800 542 0578 now!,spam
1615,"You have won ?1,000 cash or a ?2,000 prize! To...",spam
1621,Tone Club: Your subs has now expired 2 re-sub ...,spam
1622,This is the 2nd time we have tried to contact ...,spam


In [ ]:
""""TF-IDF, bigrams are used here asw"""
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # vectorizing using b-grams
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(X_train)  # training the model
clf.fit(tfidf_vectorized_x_train, y_train)  # fitting the model
tfidf_vectorized_x_test = tfidf_vectorizer.transform(X_test)  # vectorizing the test data
pred_tf_idif = clf.predict(tfidf_vectorized_x_test)
report_tf_idf = classification_report(y_test, pred)
print('metrics for TF-IDF, bigrams: ', report_tf_idf, sep='\n', end='\n')


metrics for TF-IDF, bigrams: 
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1457
        spam       0.98      0.81      0.88       211

    accuracy                           0.97      1668
   macro avg       0.97      0.90      0.93      1668
weighted avg       0.97      0.97      0.97      1668



In [ ]:
"""char n_grams"""
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 6))
char_vectorized_x_train = char_vectorizer.fit_transform(X_train)
clf.fit(char_vectorized_x_train, y_train)
char_vectorized_x_test = char_vectorizer.transform(X_test)
pred = clf.predict(char_vectorized_x_test)
report_char = classification_report(y_test, pred)
print('metrics for char n-grams: ', report_char, sep='\n', end='\n')


metrics for char n-grams: 
              precision    recall  f1-score   support

         ham       0.98      0.99      0.99      1457
        spam       0.96      0.86      0.90       211

    accuracy                           0.98      1668
   macro avg       0.97      0.93      0.95      1668
weighted avg       0.98      0.98      0.98      1668



In [ ]:
"""CONSTRUCTION OF A FINAL DF WITH BIGRAM COUNT, BIGRAM TF-IDF AND CHAR. TAG - SPAM"""
comparison_df = pd.DataFrame(columns=['Model', 'Precision', 'Recall', 'F1-Score', 'Support'])
for i in range (3):
    if i == 0:
        lines = report_count.strip().split('\n')
        model = 'CountVect'
    elif i == 1:
        lines = report_tf_idf.strip().split('\n')
        model = 'TF_IDF'
    elif i == 2:
        lines = report_char.strip().split('\n')
        model = 'Char n-grams'
    for line in lines[2:4]:
        tokens = re.split(r'\s+', line.strip())
        class_name, precision, recall, f1_score, support = tokens[0], \
                                                           float(tokens[1]), \
                                                           float(tokens[2]), \
                                                           float(tokens[3]), \
                                                           int(tokens[4])
        if class_name in ['spam']:
            comparison_df = comparison_df._append({
            'Model': model,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score,
            'Support': support
        }, ignore_index=True)

print('Final comparison DF for all models used: ', comparison_df, sep='\n')

# which model is better? e.g. has the highest average of scores
metrics_df = comparison_df.drop(columns='Model')
mean_rows = metrics_df.mean(axis=1)
print('mean values of metrics by model:', mean_rows, sep='\n', end='\n')
print('CountVectorizer and TF-IDF show the same mean values so either can be used.')

Final comparison DF for all models used: 
          Model  Precision  Recall  F1-Score Support
0     CountVect       0.98    0.81      0.88     211
1        TF_IDF       0.98    0.81      0.88     211
2  Char n-grams       0.96    0.86      0.90     211
mean values of metrics by model:
0    53.4175
1    53.4175
2    53.4300
dtype: float64
CountVectorizer and TF-IDF show the same mean values so either can be used.
